# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import math
from scipy.stats import percentileofscore as score
import xlsxwriter
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [3]:
symbol = 'AAPL'
api_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2862466476980,
 'week52high': 198.23,
 'week52low': 123.64,
 'week52highSplitAdjustOnly': 198.23,
 'week52lowSplitAdjustOnly': 124.17,
 'week52change': 0.10859801550051396,
 'sharesOutstanding': 15728702000,
 'float': 0,
 'avg10Volume': 56920945,
 'avg30Volume': 54128811,
 'day200MovingAvg': 168.25,
 'day50MovingAvg': 190.86,
 'employees': 164000,
 'ttmEPS': 5.88,
 'ttmDividendRate': 0.6980917046812681,
 'dividendYield': 0.003835879469648157,
 'nextDividendDate': '2023-08-11',
 'exDividendDate': '2023-05-12',
 'nextEarningsDate': '2023-11-02',
 'peRatio': 30.3481353779116,
 'beta': 1.291122820021239,
 'maxChangePercent': 69.85458438777498,
 'year5ChangePercent': 2.672003470435721,
 'year2ChangePercent': 0.25627828507210904,
 'year1ChangePercent': 0.10705843278143634,
 'ytdChangePercent': 0.40669903798141527,
 'month6ChangePercent': 0.1829930849604715,
 'month3ChangePercent': 0.10075284744723212,
 'month1ChangePercent': -0.05440091447573525,
 '

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [4]:
data

{'companyName': 'Apple Inc',
 'marketcap': 2862466476980,
 'week52high': 198.23,
 'week52low': 123.64,
 'week52highSplitAdjustOnly': 198.23,
 'week52lowSplitAdjustOnly': 124.17,
 'week52change': 0.10859801550051396,
 'sharesOutstanding': 15728702000,
 'float': 0,
 'avg10Volume': 56920945,
 'avg30Volume': 54128811,
 'day200MovingAvg': 168.25,
 'day50MovingAvg': 190.86,
 'employees': 164000,
 'ttmEPS': 5.88,
 'ttmDividendRate': 0.6980917046812681,
 'dividendYield': 0.003835879469648157,
 'nextDividendDate': '2023-08-11',
 'exDividendDate': '2023-05-12',
 'nextEarningsDate': '2023-11-02',
 'peRatio': 30.3481353779116,
 'beta': 1.291122820021239,
 'maxChangePercent': 69.85458438777498,
 'year5ChangePercent': 2.672003470435721,
 'year2ChangePercent': 0.25627828507210904,
 'year1ChangePercent': 0.10705843278143634,
 'ytdChangePercent': 0.40669903798141527,
 'month6ChangePercent': 0.1829930849604715,
 'month3ChangePercent': 0.10075284744723212,
 'month1ChangePercent': -0.05440091447573525,
 '

In [5]:
## Executing A Batch API Call & Building Our DataFrame

#Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

#We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [6]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#    print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [7]:
final_dataframe = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()

    for symbol in symbol_string.split(','):
        if symbol in data:
            final_dataframe = final_dataframe.append(
                pd.Series([symbol,
                           data[symbol]['quote']['latestPrice'],
                           data[symbol]['stats']['year1ChangePercent'],
                           'N/A'
                           ],
                          index=my_columns),
                ignore_index=True
            )
        else:
            print(f"Data not found for symbol: {symbol}")

final_dataframe


Data not found for symbol: DISCA
Data not found for symbol: HFC
Data not found for symbol: VIAC
Data not found for symbol: WLTW


,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,126.3,-0.049443,N/A
1,AAL,15.84,0.065949,N/A
2,AAP,71.01,-0.615968,N/A
3,AAPL,181.99,0.107058,N/A
4,ABBV,147.73,0.106559,N/A
...,...,...,...,...
496,YUM,134.35,0.135314,N/A
497,ZBH,126.32,0.123035,N/A
498,ZBRA,251.41,-0.239488,N/A
499,ZION,37.92,-0.259434,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [8]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:51]
final_dataframe.reset_index(inplace = True)
final_dataframe


,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,273,LB,79.92,2.281287,N/A
1,388,RCL,103.92,1.634221,N/A
2,343,NVDA,446.8,1.32743,N/A
3,193,FTI,18.31,1.297365,N/A
4,368,PHM,84.35,1.003739,N/A
5,197,GE,113.06,0.957583,N/A
6,330,NFLX,431.6,0.877256,N/A
7,84,CCL,17.16,0.76,N/A
8,410,SLB,57.67,0.721857,N/A
9,132,DHI,125.71,0.655072,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [9]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the size of your portfolio: ')
    
    try:
        float(portfolio_size)
    except ValueError:
        print('That is not a number! ')
        print("Please try again")
        portfolio_size = input('Enter the size of your portfolio: ')

portfolio_input()
print(portfolio_size)

Enter the size of your portfolio: 1000000
1000000


In [10]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for i in range(0, len(final_dataframe)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Price'])
    
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,273,LB,79.92,2.281287,245
1,388,RCL,103.92,1.634221,188
2,343,NVDA,446.8,1.32743,43
3,193,FTI,18.31,1.297365,1070
4,368,PHM,84.35,1.003739,232
5,197,GE,113.06,0.957583,173
6,330,NFLX,431.6,0.877256,45
7,84,CCL,17.16,0.76,1142
8,410,SLB,57.67,0.721857,340
9,132,DHI,125.71,0.655072,155


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [11]:
hqm_columns = [
    'Tickers',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]
hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch/?types=stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        if symbol in data:
            hqm_dataframe = hqm_dataframe.append(
                pd.Series(
            [
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['stats']['year1ChangePercent'],
                'N/A',
                data[symbol]['stats']['month6ChangePercent'],
                'N/A',
                data[symbol]['stats']['month3ChangePercent'],
                'N/A',
                data[symbol]['stats']['month1ChangePercent'],
                'N/A',
                'N/A'
            ],
                index = hqm_columns),
                ignore_index = True
            )
        else:
            print(f"Data not found for symbol: {symbol}")
        
hqm_dataframe

Data not found for symbol: DISCA
Data not found for symbol: HFC
Data not found for symbol: VIAC
Data not found for symbol: WLTW


,Tickers,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,126.3,N/A,-0.049443,N/A,-0.179894,N/A,-0.054816,N/A,0.058498,N/A,N/A
1,AAL,15.84,N/A,0.065949,N/A,-0.06933,N/A,0.172465,N/A,-0.124862,N/A,N/A
2,AAP,71.01,N/A,-0.615968,N/A,-0.525834,N/A,-0.407337,N/A,0.004395,N/A,N/A
3,AAPL,181.99,N/A,0.107058,N/A,0.182993,N/A,0.100753,N/A,-0.054401,N/A,N/A
4,ABBV,147.73,N/A,0.106559,N/A,0.038167,N/A,0.013625,N/A,0.106346,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,134.35,N/A,0.135314,N/A,0.062727,N/A,-0.009853,N/A,-0.021272,N/A,N/A
497,ZBH,126.32,N/A,0.123035,N/A,-0.020692,N/A,-0.073922,N/A,-0.113606,N/A,N/A
498,ZBRA,251.41,N/A,-0.239488,N/A,-0.240017,N/A,-0.051283,N/A,-0.151416,N/A,N/A
499,ZION,37.92,N/A,-0.259434,N/A,-0.285274,N/A,0.936067,N/A,0.347548,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [12]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        
        hqm_dataframe[change_col] = hqm_dataframe[change_col].fillna(0)
        hqm_dataframe.loc[row, percentile_col] = score(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])/100
        
        
hqm_dataframe

,Tickers,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,126.3,N/A,-0.049443,0.291417,-0.179894,0.091816,-0.054816,0.149701,0.058498,0.766467,N/A
1,AAL,15.84,N/A,0.065949,0.518962,-0.069330,0.291417,0.172465,0.750499,-0.124862,0.047904,N/A
2,AAP,71.01,N/A,-0.615968,0.003992,-0.525834,0.003992,-0.407337,0.003992,0.004395,0.50499,N/A
3,AAPL,181.99,N/A,0.107058,0.618762,0.182993,0.898204,0.100753,0.578842,-0.054401,0.159681,N/A
4,ABBV,147.73,N/A,0.106559,0.616766,0.038167,0.60479,0.013625,0.373253,0.106346,0.892216,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,134.35,N/A,0.135314,0.668663,0.062727,0.684631,-0.009853,0.279441,-0.021272,0.329341,N/A
497,ZBH,126.32,N/A,0.123035,0.654691,-0.020692,0.43513,-0.073922,0.111776,-0.113606,0.055888,N/A
498,ZBRA,251.41,N/A,-0.239488,0.065868,-0.240017,0.057884,-0.051283,0.157685,-0.151416,0.021956,N/A
499,ZION,37.92,N/A,-0.259434,0.055888,-0.285274,0.02994,0.936067,1.0,0.347548,1.0,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [13]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
        
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

hqm_dataframe

,Tickers,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,126.3,N/A,-0.049443,0.291417,-0.179894,0.091816,-0.054816,0.149701,0.058498,0.766467,0.32485
1,AAL,15.84,N/A,0.065949,0.518962,-0.069330,0.291417,0.172465,0.750499,-0.124862,0.047904,0.402196
2,AAP,71.01,N/A,-0.615968,0.003992,-0.525834,0.003992,-0.407337,0.003992,0.004395,0.50499,0.129242
3,AAPL,181.99,N/A,0.107058,0.618762,0.182993,0.898204,0.100753,0.578842,-0.054401,0.159681,0.563872
4,ABBV,147.73,N/A,0.106559,0.616766,0.038167,0.60479,0.013625,0.373253,0.106346,0.892216,0.621756
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,134.35,N/A,0.135314,0.668663,0.062727,0.684631,-0.009853,0.279441,-0.021272,0.329341,0.490519
497,ZBH,126.32,N/A,0.123035,0.654691,-0.020692,0.43513,-0.073922,0.111776,-0.113606,0.055888,0.314371
498,ZBRA,251.41,N/A,-0.239488,0.065868,-0.240017,0.057884,-0.051283,0.157685,-0.151416,0.021956,0.075848
499,ZION,37.92,N/A,-0.259434,0.055888,-0.285274,0.02994,0.936067,1.0,0.347548,1.0,0.521457


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [14]:
hqm_dataframe.sort_values('HQM Score', ascending = False, inplace = True)
hqm_dataframe = hqm_dataframe[:51]
hqm_dataframe.reset_index(drop = True, inplace = True)
hqm_dataframe

,Tickers,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,FTI,18.31,N/A,1.297365,0.994012,0.393455,0.988024,0.375657,0.968064,0.091180,0.860279,0.952595
1,ANET,179.6,N/A,0.459451,0.938124,0.361018,0.98004,0.327028,0.94012,0.123202,0.914172,0.943114
2,MPC,137.95,N/A,0.568562,0.972056,0.181947,0.892216,0.322021,0.934132,0.173643,0.966068,0.941118
3,CARR,57.08,N/A,0.388470,0.916168,0.227532,0.926148,0.394157,0.974052,0.140687,0.946108,0.940619
4,PHM,84.35,N/A,1.003739,0.992016,0.441757,0.99002,0.273661,0.9002,0.091344,0.862275,0.936128
5,NVDA,446.8,N/A,1.327430,0.996008,1.118334,1.0,0.621408,0.99002,0.053451,0.752495,0.934631
6,BKR,35.52,N/A,0.549813,0.964072,0.147813,0.856287,0.322811,0.936128,0.124051,0.92016,0.919162
7,LB,79.92,N/A,2.281287,1.0,0.815831,0.998004,0.215421,0.850299,0.079271,0.828343,0.919162
8,ETN,215.9,N/A,0.487951,0.9501,0.341253,0.974052,0.293463,0.914172,0.080390,0.834331,0.918164
9,COG,22.25,N/A,0.340483,0.884232,0.180873,0.888224,0.255629,0.89022,0.249298,0.994012,0.914172


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [15]:
portfolio_input()

Enter the size of your portfolio: 1000000


In [16]:
position_size = float(portfolio_size)/len(hqm_dataframe.index)
for i in hqm_dataframe.index:
    hqm_dataframe.loc[i, 'Numver of Shares to Buy'] = math.floor(position_size/hqm_dataframe.loc[i, 'Price'])
    
hqm_dataframe

/var/folders/gm/l0x17nk537n3csw3nq5h9gb80000gn/T/ipykernel_21501/4014512624.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hqm_dataframe.loc[i, 'Numver of Shares to Buy'] = math.floor(position_size/hqm_dataframe.loc[i, 'Price'])


,Tickers,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score,Numver of Shares to Buy
0,FTI,18.31,N/A,1.297365,0.994012,0.393455,0.988024,0.375657,0.968064,0.091180,0.860279,0.952595,1070.0
1,ANET,179.6,N/A,0.459451,0.938124,0.361018,0.98004,0.327028,0.94012,0.123202,0.914172,0.943114,109.0
2,MPC,137.95,N/A,0.568562,0.972056,0.181947,0.892216,0.322021,0.934132,0.173643,0.966068,0.941118,142.0
3,CARR,57.08,N/A,0.388470,0.916168,0.227532,0.926148,0.394157,0.974052,0.140687,0.946108,0.940619,343.0
4,PHM,84.35,N/A,1.003739,0.992016,0.441757,0.99002,0.273661,0.9002,0.091344,0.862275,0.936128,232.0
5,NVDA,446.8,N/A,1.327430,0.996008,1.118334,1.0,0.621408,0.99002,0.053451,0.752495,0.934631,43.0
6,BKR,35.52,N/A,0.549813,0.964072,0.147813,0.856287,0.322811,0.936128,0.124051,0.92016,0.919162,552.0
7,LB,79.92,N/A,2.281287,1.0,0.815831,0.998004,0.215421,0.850299,0.079271,0.828343,0.919162,245.0
8,ETN,215.9,N/A,0.487951,0.9501,0.341253,0.974052,0.293463,0.914172,0.080390,0.834331,0.918164,90.0
9,COG,22.25,N/A,0.340483,0.884232,0.180873,0.888224,0.255629,0.89022,0.249298,0.994012,0.914172,881.0


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [17]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine = 'xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name = 'Momentum Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [18]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [19]:
column_formats = { 
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['One-Year Price Return', percent_template],
                    'E': ['One-Year Return Percentile', percent_template],
                    'F': ['Six-Month Price Return', percent_template],
                    'G': ['Six-Month Return Percentile', percent_template],
                    'H': ['Three-Month Price Return', percent_template],
                    'I': ['Three-Month Return Percentile', percent_template],
                    'J': ['One-Month Price Return', percent_template],
                    'K': ['One-Month Return Percentile', percent_template],
                    'L': ['HQM Score', integer_template]
                    }

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], string_template)

## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [20]:
writer.save()